<a href="https://colab.research.google.com/github/HammadN98/pytorch/blob/main/07_pytorch_experimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>